#### @prompt example

In [1]:
from openbb import obb
import os
from magentic import prompt
from openbb import obb
obb.account.login(pat=os.environ['PAT_KEY'])
FMP_KEY = os.environ['FMP_KEY']


## Six Step Dance via Langchain
1. Taking into Account Company Industry (Growth / Value)
    1. Growth / Value
    2. Sector outperforming market or not
    3. Company outperforming sector
    4. PE vs Industry
2. Determining Company Financial Stability
    1. Debt To Equity
    2. Current Ratio
    3. Interest Margin Cover
3. Looking at Historical Earning and Growth
    1. Sales Growth
    2. EPS Growth
    3. Return on Invested Capital
    4. Profit Margin
    5. Cashflow Growth
    6. Equity Growth
    7. Return on Equity
4. Understanding Earnings and Sales Expectations
    1. Analyst Estimates
    2. PEG
5. Checking out competition
    1. P/E
    2. P/S
    3.MOAT
6. Estimating Company Values
    1. Insider Buying
    2. Senate TRading
    3. 13f

## Buffett Criteria
### Gross Margin Indicates profitability > 40% GrossProfit/Revenue GROSS PM close to 40 for recent years
### SG&A Margin efficiency in managing overhead costs < 30% SG&A Expenses / Gross Profit
### R&D Margin r&d spending relative to profitability < 30% R&D Expenses / Gross Profit
### Interest Expense Margin reliance on debt and financial health < 15 % Interest Expense / Operating Income
### Income Tax Margin Gauges tax efficiency ~ 20% Income Tax / Pre-Tax Income
### Profit Margin Shows overall profitabilty relative to revenue > 20% Net Income / Revenue
### EPS Growth Indicates growth and consistency in earnings >0 and growing Net Income / Shares Outstanding




In [ ]:
obb.news.world(provider='fmp').to_llm()

In [ ]:



@prompt("Explain like I'm five this financial concept: {concept}")
def explain(concept: str) -> str: ...


explain("Subprime mortgage crisis")

In [ ]:

from magentic import prompt
from pydantic import BaseModel


class Portfolio(BaseModel):
    equity_etf_pct: float
    bond_etf_pc: float
    crypto_etf_pc: float
    commodities_pc: float
    reasoning: str


@prompt("Create a strong portfolio of {size} allocation size.")
def create_portfolio(size: str) -> Portfolio: ...


portfolio = create_portfolio("$50,000")

#### @chatprompt decorator

In [ ]:
from magentic import chatprompt, AssistantMessage, SystemMessage, UserMessage
from pydantic import BaseModel

class Quote(BaseModel):
    quote: str
    person: str


@chatprompt(
    SystemMessage("You are an avid reader of financial literature."),
    UserMessage("What is your favorite quote from Warren Buffet?"),
    AssistantMessage(
        Quote(
            quote="Price is what you pay; value is what you get.",
            person="Warren Buffet",
        )
    ),
    UserMessage("What is your favorite quote from {person}?"),
)
def get_finance_quote(person: str) -> Quote: ...


get_finance_quote("Charlie Munger")

#### Function calling

In [ ]:
import os
import requests
from magentic import prompt, FunctionCall
import logging
from openbb import obb
from langchain.tools import Tool
import reticker


def get_income_statement(query:str) -> str:
    ''' retrieves income statement for a company'''
    try:
        ticker = reticker(query)
        period = 'annual' if 'annual' in query else 'quarterly'
        return obb.equity.fundamental.income(symbol=ticker, period=period, limit=5, provider='fmp').to_llm()
    except Exception as e:
        return f'Unable to find data for {ticker}'

def get_balance_sheet(query:str) -> str:
    ''' retrieves balance sheet for a company'''
    try:
        ticker = reticker(query)
        period = 'annual' if 'annual' in query else 'quarterly'
        return obb.equity.fundamental.balance(symbol=ticker, period=period, limit=5, provider='intrinio')
    except Exception as e:
        return f'Unable to find data for {ticker}'


def get_aggressive_smallcaps() -> list :
    ''' retrieves aggressive smallcaps'''
    return obb.equity.discovery.aggressive_small_caps(sort='desc').to_llm()


def get_undervalued_growth() -> list:
    ''' retrieves undervalued growth stocks'''
    return obb.equity.discovery.undervalued_growth(provider='yfinance').to_llm()


def get_ticker_from_query(query):
    extractor = reticker.TickerExtractor(deduplicate=True)
    tickers = extractor.extract(query)
    if len(tickers) > 1:
        return ','.join(tickers)
    return tickers[0] 


def get_stock_price(query: str) -> str:
    """Get the current stock price for a given ticker."""
    try:
        ticker = get_ticker_from_query(query)
        data = obb.equity.price.quote(symbol=ticker)

        return data.to_llm()
    except Exception as e:
        return f"Error fetching stock price for {ticker}: {str(e)}"

def get_company_overview(query: str) -> str:
    """Get an overview of a company for a given ticker."""
    try:
        ticker = get_ticker_from_query(query)
        data = obb.equity.profile(symbol=ticker)
        return data.to_llm()
    except Exception as e:
        return f"Error fetching company overview for {ticker}: {str(e)}"
    
def get_group_performance(group:str) -> list:
    """ Get performane by industry or sector"""
    data = obb.equity.compare.groups(group=group, metric='performance', provider='finviz')
    return data.to_llm()

def get_latest_news_for_company(query : str) -> str:
    """ Get latest news for a company """
    try:
        ticker = get_ticker_from_query(query)
        return obb.news.company(symbol=ticker)
    except Exception as e:
        return f"Error fetching stock price for {ticker}: {str(e)}"

@prompt(
    "Use the appropriate search function to answer: {question}",
    functions=[get_stock_price, get_aggressive_smallcaps, get_undervalued_growth, get_latest_news_for_company,
               get_balance_sheet, get_income_statement, get_group_performance],
)
def perform_search(question: str) -> FunctionCall[str]: ...


output = perform_search("Find the best performing industry")
result = output()
from pprint import pprint
pprint(result)

#### Prompt Chains

In [ ]:
import csv
from magentic import prompt_chain


def get_earnings_calendar(ticker: str, api_key: str = FMP_KEY) -> list:
    url = f"https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol={ticker}&horizon=12month&apikey={api_key}"
    with requests.Session() as s:
        download = s.get(url)
        decoded_content = download.content.decode("utf-8")
        cr = csv.reader(decoded_content.splitlines(), delimiter=",")
        my_list = list(cr)
    return my_list


@prompt_chain(
    "What's {ticker} expected earnings dates for the next 12 months?",
    functions=[get_earnings_calendar],
)
def get_earnings(ticker: str) -> str: ...


get_earnings("IBM")

#### Streaming Response

In [ ]:
from magentic import StreamedStr


@prompt("Explain to me {term} in a way a 5-year-old would understand.")
def describe_finance_term(term: str) -> StreamedStr: ...


# Print the chunks while they are being received
for chunk in describe_finance_term("liquidity"):
    print(chunk, end="")

#### Streaming Structured Outputs

In [ ]:
from collections.abc import Iterable
from time import time


class Portfolio(BaseModel):
    equity_etf_pct: float
    bond_etf_pc: float
    crypto_etf_pc: float
    commodities_pc: float
    reasoning: str


@prompt("Create {n_portfolio} portfolios with varying deegress of risk apetite.")
def create_portfolios(n_portfolio: int) -> Iterable[Portfolio]: ...


start_time = time()
for portfolio in create_portfolios(3):
    print(f"{time() - start_time:.2f}s : {portfolio}")

#### Asynchronous Streaming

In [ ]:
import asyncio
from typing import AsyncIterable


@prompt("List three high-growth stocks.")
async def iter_growth_stocks() -> AsyncIterable[str]: ...


@prompt("Tell me more about {stock_symbol}")
async def tell_me_more_about(stock_symbol: str) -> str: ...


start_time = time()
tasks = []
async for stock in await iter_growth_stocks():
    # Use asyncio.create_task to schedule the coroutine for execution before awaiting it
    # This way descriptions will start being generated while the list of stocks is still being generated
    task = asyncio.create_task(tell_me_more_about(stock))
    tasks.append(task)

descriptions = await asyncio.gather(*tasks)

for desc in descriptions:
    print(desc)

### Attempting to write a Small Agent

In [2]:
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [ ]:
@tool
def get_performance(group:str) -> list:
    """ Return performance by sector or industyr industry performance for last week, last month, last quarter, last half year and last year"""
    return obb.equity.compare.groups(group=group, metric='performance', provider='finviz').to_llm()

@tool
def get_strong_buy_for_sector(sector : str) -> list :
    """ Return the strong buy recommendation for a given sector"""
    new_sector = '_'.join(sector.lower().split())
    data = obb.equity.screener(provider='finviz', sector=new_sector, recommendation='buy')
    return data.to_llm()

@tool
def get_best_stock_performers_for_sector(sector:str) -> list :
    """ Return the best  10 stock performers for last week and last month for a sector"""
    data = obb.equity.screener(provider='finviz', filters_dict={'Sector' : sector, 'Performance' : 'Week Up', 'Performance 2' : 'Month Up'}, limit=10)
    return data.to_llm()

@tool
def get_best_stock_performers_for_industry(industry:str) -> list :
    """ Return the best  10 stock performers for last week and last month for a sector"""
    data = obb.equity.screener(provider='finviz', filters_dict={'Sector' : industry, 'Performance' : 'Week Up', 'Performance 2' : 'Month Up'}, limit=10)
    return data.to_llm()

@tool
def get_valuation_for_sectors(input:str) -> list:
    """ Return valuation meetrics for all sectors"""
    data = obb.equity.compare.groups(group='sector', metric='valuation', provider='finviz').to_df()
    
    filtered =  data[data.name == input]
    return filtered.to_json(
            orient="records",
            date_format="iso",
            date_unit="s",
        )

@tool
def get_valuation_for_industries() -> list:
    """ Return valuation meetrics for all sectors"""
    return obb.equity.compare.groups(group='industry', metric='valuation', provider='finviz').to_llm()


@tool
def get_valuation_for_company(ticker:str) -> list:
    """ Return Valuation Ratios  for a company"""#
    # requires obb login
    obb.account.login(pat=os.environ['PAT_KEY'])
    return obb.equity.fundamental.ratios_ttm(symbol='AAPL', provider='fmp', limit=1).to_llm()

#get_strong_buy_for_sector.invoke('Consumer Cyclical')
get_valuation_for_sectors.invoke('Financials')

'[]'

In [ ]:
data = obb.equity.compare.groups(group='sector', metric='valuation', provider='finviz').to_df()

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful stock financial analys, but don't know current events. Use the toools as much as you can to retrieve informations",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [8]:
tools = [get_performance, get_strong_buy_for_sector, get_best_stock_performers_for_sector, get_valuation_for_group,get_valuation_for_company]
llm_with_tools = llm.bind_tools(tools)

In [21]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [22]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [23]:
list(agent_executor.stream({"input": "What is the best performing sector for last month"}))



> Entering new None chain...

Invoking: `get_performance` with `{'group': 'sector'}`


[{"name":"Consumer Defensive","performance_1d":0.0056,"performance_1w":-0.0112,"performance_1m":-0.0323,"performance_3m":-0.0033,"performance_6m":-0.0123,"performance_1y":0.07,"performance_ytd":0.0149,"analyst_recommendation":2.01,"volume":447610000,"volume_average":466650000,"volume_relative":0.96},{"name":"Basic Materials","performance_1d":0.0071,"performance_1w":-0.0057,"performance_1m":0.0043,"performance_3m":0.0583,"performance_6m":-0.0498,"performance_1y":-0.0265,"performance_ytd":0.0692,"analyst_recommendation":2.03,"volume":656000000,"volume_average":648590000,"volume_relative":1.01},{"name":"Real Estate","performance_1d":0.0144,"performance_1w":-0.0042,"performance_1m":-0.004,"performance_3m":0.0287,"performance_6m":-0.0644,"performance_1y":0.0699,"performance_ytd":0.0237,"analyst_recommendation":1.97,"volume":319170000,"volume_average":376220000,"volume_relative":0.85},{"name":"Utilities"

[{'actions': [ToolAgentAction(tool='get_performance', tool_input={'group': 'sector'}, log="\nInvoking: `get_performance` with `{'group': 'sector'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_PGwnZQ0GxKiUxm8tjErzQzOB', 'function': {'arguments': '{"group":"sector"}', 'name': 'get_performance'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-71913799-e284-4612-8d70-f3ee5fb9d49c', tool_calls=[{'name': 'get_performance', 'args': {'group': 'sector'}, 'id': 'call_PGwnZQ0GxKiUxm8tjErzQzOB', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_performance', 'args': '{"group":"sector"}', 'id': 'call_PGwnZQ0GxKiUxm8tjErzQzOB', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_PGwnZQ0GxKiUxm8tjErzQzOB')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_PGwnZQ0GxKiUxm8tjErzQzOB', 'function': {'argume

### Chat Memory

In [25]:
from langchain_core.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful stock recommendation assistant , but dont know current events so you should use your tools as much as you can.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [26]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

In [27]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [32]:
input1 = "Which sector shown a constant postive performance across quarter, month and week?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `get_performance` with `{'group': 'sector'}`


[{"name":"Consumer Defensive","performance_1d":0.0056,"performance_1w":-0.0112,"performance_1m":-0.0323,"performance_3m":-0.0033,"performance_6m":-0.0123,"performance_1y":0.07,"performance_ytd":0.0149,"analyst_recommendation":2.01,"volume":447610000,"volume_average":466650000,"volume_relative":0.96},{"name":"Basic Materials","performance_1d":0.0071,"performance_1w":-0.0057,"performance_1m":0.0043,"performance_3m":0.0583,"performance_6m":-0.0498,"performance_1y":-0.0265,"performance_ytd":0.0692,"analyst_recommendation":2.03,"volume":656000000,"volume_average":648590000,"volume_relative":1.01},{"name":"Real Estate","performance_1d":0.0144,"performance_1w":-0.0042,"performance_1m":-0.004,"performance_3m":0.0287,"performance_6m":-0.0644,"performance_1y":0.0699,"performance_ytd":0.0237,"analyst_recommendation":1.97,"volume":319170000,"volume_average":376220000,"volume_relative":0.85},{"name":"U

In [31]:
input1 = "What are the valutiom metrics for this sector"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `get_valuation_for_group` with `{'group': 'Financial'}`


[{"name":"Financial Conglomerates","market_cap":41470000000,"performance_1d":0.012,"pe":11.4,"forward_pe":2.76,"peg":0.86,"eps_growth_past_5y":0.2385,"eps_growth_next_5y":0.132,"volume":1940000,"price_to_sales":1.26,"price_to_book":1.24,"price_to_cash":4.22,"price_to_free_cash_flow":4.03,"sales_growth_past_5_years":0.1758},{"name":"Coking Coal","market_cap":5530000000,"performance_1d":0.0329,"pe":10.22,"forward_pe":6.01,"peg":0.56,"eps_growth_past_5y":-0.0269,"eps_growth_next_5y":0.181,"volume":5310000,"price_to_sales":0.78,"price_to_book":1.15,"price_to_cash":4.57,"price_to_free_cash_flow":10.62,"sales_growth_past_5_years":0.1487},{"name":"Insurance - Reinsurance","market_cap":45630000000,"performance_1d":0.023,"pe":10.67,"forward_pe":6.64,"peg":0.67,"eps_growth_past_5y":0.0403,"eps_growth_next_5y":0.1588,"volume":2620000,"price_to_sales":0.8,"price_to_book":1.17,"price_to_cash

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 17583 tokens (17405 in the messages, 178 in the functions). Please reduce the length of the messages or functions.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [ ]:
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "Which stock are recommented for this sector best performing sector", "chat_history": chat_history})